In [ ]:
def clean_vehicles_dataset(df):
    """
    Thorough dataset cleaning
    """
    # Cleaning: Setting vehicle guzzler class to binary
    df['guzzler'] = np.where(((df.guzzler == 'G')
                            | (df.guzzler == 'T')
                            | (df.guzzler == 'S')),
                                1,
                                0)

    # Cleaning: imputing cylinders to 4
    df['cylinders'] = np.where((df.make == 'Subaru') 
                             & (df.model == 'RX Turbo'),
                               4,
                               df.cylinders) 

    df.drop(index=21500, inplace=True) # Cleaning: dropping duplicate data entry

    # Cleaning: Sets cylinders to 0 for all electric vehicles with NaN values
    df['cylinders'] = np.where(df.cylinders.isna(),
                               0,
                               df.cylinders)

    # Cleaning: Sets displacement to 0 for all electric vehicles with NaN values
    df['displ'] = np.where(df.displ.isna(),
                          0,
                          df.displ)

    # Cleaning: Setting to dummy binary class while simultaneously managing NaN values
    df['startStop'] = np.where((df.startStop == 'Y'),
                              1,
                              0)

    # Cleaning: Setting from boolean to binary class
    df['phevBlended'] = np.where(df.phevBlended == True,
                                1,
                                0)

    # Cleaning: Setting class names to consistent categories 
    df['drive'] = np.where(((df.drive == '4-Wheel or All-Wheel Drive')
                          | (df.drive == 'All-Wheel Drive')
                          | (df.drive == '4-Wheel Drive')),
                          '4WD',
                          df.drive)

    df['drive'] = np.where(df.drive == 'Front-Wheel Drive',
                          'FWD',
                           df.drive)

    df['drive'] = np.where(df.drive == 'Rear-Wheel Drive',
                          'RWD',
                           df.drive)

    # Cleaning: electric truck that has FWD, unlike other vehicles that listed 2WD which are RWD
    df['drive'] = np.where(df.model == 'S10 Electric',
                           'FWD',
                           df.drive)

    # Cleaning: setting the other vehicles listed w/ 2WD to RWD
    df['drive'] = np.where(df.drive == '2-Wheel Drive',
                          'RWD',
                           df.drive)

    # Cleaning: setting part time 4WD to pt4WD, for easier manipulation and consistency with other classes
    df['drive'] = np.where(df.drive == 'Part-time 4-Wheel Drive',
                          'pt4WD',
                           df.drive)

    # Cleaning: combining similar vehicle classes 
    df['VClass'] = np.where(((df.VClass == 'Sport Utility Vehicle - 4WD')
                           | (df.VClass == 'Sport Utility Vehicle - 2WD')
                           | (df.VClass == 'Small Sport Utility Vehicle 4WD')
                           | (df.VClass == 'Standard Sport Utility Vehicle 4WD')
                           | (df.VClass == 'Small Sport Utility Vehicle 2WD')
                           | (df.VClass == 'Standard Sport Utility Vehicle 2WD')),
                             'SUV',
                              df.VClass)

    df['VClass'] = np.where(((df.VClass == 'Standard Pickup Trucks')
                           | (df.VClass == 'Standard Pickup Trucks 2WD')
                           | (df.VClass == 'Standard Pickup Trucks 4WD')
                           | (df.VClass == 'Small Pickup Trucks')
                           | (df.VClass == 'Small Pickup Trucks 2WD')
                           | (df.VClass == 'Small Pickup Trucks 4WD')
                           | (df.VClass == 'Standard Pickup Trucks/2wd')),
                             'Pickup Trucks',
                              df.VClass)

    df['VClass'] = np.where(((df.VClass == 'Small Station Wagons')
                           | (df.VClass == 'Midsize-Large Station Wagons')
                           | (df.VClass == 'Midsize Station Wagons')),
                             'Station Wagons',
                              df.VClass)

    df['VClass'] = np.where(((df.VClass == 'Special Purpose Vehicles')
                           | (df.VClass == 'Special Purpose Vehicles 2WD')
                           | (df.VClass == 'Special Purpose Vehicles 4WD')
                           | (df.VClass == 'Special Purpose Vehicles/4wd')
                           | (df.VClass == 'Special Purpose Vehicles/2wd')
                           | (df.VClass == 'Special Purpose Vehicle')
                           | (df.VClass == 'Special Purpose Vehicle 2WD')
                           | (df.VClass == 'Special Purpose Vehicle 4WD')),
                             'Special Purpose Vehicles',
                              df.VClass)

    df['VClass'] = np.where(((df.VClass == 'Vans, Cargo Type')
                           | (df.VClass == 'Minivan - 2WD')
                           | (df.VClass == 'Vans, Passenger Type')
                           | (df.VClass == 'Minivan - 4WD')
                           | (df.VClass == 'Vans Passenger')),
                             'Vans',
                              df.VClass)

    # Cleaning: Imputing displacement based on median for Suburus with 4 cylinders
    df['displ'] = np.where(((df.displ == 0)
                          & (df.fuelType1 != 'Electricity')),
                             2.2, # median for Subarus w/ 4 cylinders
                             df.displ)

    # OHE: Establishing feature for pure BEVs (since I'm dropping eng_dscr feature)
    df['BEV'] = np.where(df.displ == 0,
                        1,
                        0)

    # OHE: Establishing feature for PHEVs (again, in lieu of other temp-dropped features)
    df['PHEV'] = np.where(df.fuelType2 == 'Electricity',
                          1,
                          0)

    # OHE: Establishing feature for ICEs (again, in lieu of other temp-dropped features)
    df['ICE'] = np.where(((df.fuelType1 != 'Electricity')
                        & (df.fuelType2 != 'Electricity')),
                          1,
                          0)

    # FE: Creating target variable feature, and inputting value for ICE vehicles that have 0 for unrounded MPG value
    df['y'] = np.where(((df.ICE == 1) 
                      & (df.comb08U == 0)),
                         df.comb08,
                          0)

    # FE: Inputting target value for ICE vehicles that have rounded MPG values
    df['y'] = np.where(((df.ICE == 1)
                      & (df.y == 0)),
                         df.comb08U,
                         df.y)

    # FE: Inputting target value for PHEVs
    df['y'] = np.where(df.PHEV == 1,
                       df.phevComb,
                       df.y)

    # FE: Inputting target value for BEVs
    df['y'] = np.where(df.BEV == 1,
                      df.comb08,
                      df.y)

    # OHE: Setting null values and 'T' class to binary
    df['tCharger'] = np.where(df.tCharger == 'T',
                             1,
                             0)

    # OHE: Setting null values and 'S' class to binary
    df['sCharger'] = np.where(df.sCharger == 'S',
                             1,
                             0)

    # Cleaning: Removing vehicles with no transmission that I'm unable to impute
    df = df.drop(7138) # 2000 Nissan Altra EV
    df = df.drop(8144) # 2001 Ford Th!nk
    df = df.drop(8146) # 2001 Ford Explorer USPS Electric
    df = df.drop(9213) # 2002 Ford Explorer USPS Electric
    df = df.drop(8147) # 2001 Nissan Hyper-Mini

    # Cleaning: Removing vehicles with no drivetrain information that I'm unable to impute
    df = df.drop(18229) # 1984 Kenyon Corporation of America Kenyon 5.0 Cabrio



    # Cleaning: Imputting missing values from other vehicles of same make/model/year
    df['trany'] = np.where(((df.model == 'RAV4 EV')
                          & (df.trany.isna() == True)),
                            'Automatic (variable gear ratios)',
                             df.trany)

    # Cleaning: Imputting missing values from other vehicles of same make/model/year
    df['trany'] = np.where(((df.model == 'F150 Pickup 2WD') 
                          & (df.year == 1984) 
                          & (df.cylinders == 8) 
                          & (df.displ == 5.8)
                          & (df.trany.isna() == True)),
                            'Automatic 3-spd', # imputing from most common for those fitting the parameters above
                           df.trany)

    df['trany'] = np.where(((df.model == 'C25 Pickup 2WD') 
                          & (df.year == 1984)
                          & (df.trany.isna() == True)),
                            'Automatic 4-spd',
                             df.trany)

    # Cleaning: Renaming classes to consistent values
    # for automatic transmissions:
    df['trany'] = np.where(((df.trany == 'Automatic 3-spd')
                          | (df.trany == 'Automatic (L3)')
                          | (df.trany == '(Automatic (L3)')),
                           'A3S',
                           df.trany)

    df['trany'] = np.where(((df.trany == 'Automatic 4-spd')
                          | (df.trany == 'Automatic (S4)')
                          | (df.trany == 'Automatic (L4)')),
                            'A4S',
                             df.trany)

    df['trany'] = np.where(((df.trany == 'Automatic 5-spd')
                          | (df.trany == 'Automatic (S5)')
                          | (df.trany == 'Automatic (AM5)')),
                           'A5S',
                           df.trany)

    df['trany'] = np.where(((df.trany == 'Automatic (S6)')
                          | (df.trany == 'Automatic 6-spd')
                          | (df.trany == 'Automatic (AV-S6)')
                          | (df.trany == 'Automatic (AM6)')
                          | (df.trany == 'Automatic (AM-S6)')
                          | (df.trany == 'Automatic (AV-S1)')),
                           'A6S',
                           df.trany)

    df['trany'] = np.where(((df.trany == 'Automatic 7-spd')
                          | (df.trany == 'Automatic (S7)')
                          | (df.trany == 'Automatic (AM7)')
                          | (df.trany == 'Automatic (AM-S7)')
                          | (df.trany == 'Automatic (AV-S7)')),
                           'A7S',
                             df.trany)

    df['trany'] = np.where(((df.trany == 'Automatic 8-spd')
                          | (df.trany == 'Automatic (S8)')
                          | (df.trany == 'Automatic (AM8)')
                          | (df.trany == 'Automatic (AM-S8)')
                          | (df.trany == 'Automatic (AV-S8)')),
                           'A8S',
                             df.trany)

    df['trany'] = np.where(((df.trany == 'Automatic 9-spd')
                          | (df.trany == 'Automatic (S9)')
                          | (df.trany == 'Automatic (AM9)')
                          | (df.trany == 'Automatic (AM-S9)')
                          | (df.trany == 'Automatic (AV-S9)')),
                           'A9S',
                             df.trany)

    df['trany'] = np.where(((df.trany == 'Automatic 10-spd')
                          | (df.trany == 'Automatic (S10)')
                          | (df.trany == 'Automatic (AM10)')
                          | (df.trany == 'Automatic (AM-S10)')
                          | (df.trany == 'Automatic (AV-S10)')),
                           'A10S',
                             df.trany)

    # special electric car transmissions
    df['trany'] = np.where(df.trany == 'Automatic (A1)',
                           'A1E',
                           df.trany)

    df['trany'] = np.where(df.trany == 'Automatic (A2)',
                           'A2E',
                           df.trany)

    # ..and again for manual:
    df['trany'] = np.where(df.trany == 'Manual 3-spd',
                           'M3S',
                           df.trany)

    df['trany'] = np.where(((df.trany == 'Manual 4-spd')
                          | (df.trany == 'Manual 4-spd Doubled')),
                           'M4S',
                           df.trany)

    df['trany'] = np.where(df.trany == 'Manual 5-spd',
                           'M5S',
                           df.trany)

    df['trany'] = np.where(df.trany == 'Manual 6-spd',
                           'M6S',
                           df.trany)

    df['trany'] = np.where(df.trany == 'Manual 7-spd',
                           'M7S',
                           df.trany)

    # Cleaning: Imputing Null Values
    df['drive'] = np.where(((df.drive.isna())
                          & (df.model == 'Escort')),
                          'FWD',
                          df.drive)

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == 'Spider Veloce 2000')),
                            'RWD',
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == 'Corvette')),
                            'RWD', 
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == '300ZX')),
                            'RWD', 
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == 'Civic CRX')),
                            'FWD', 
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == 'Countach Lp500s')),
                            'RWD', 
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == 'RX-7')),
                            'RWD', 
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == '380SL')),
                            'RWD', 
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == 'Spider')
                          & (df.make == 'Pininfarina')),
                            'RWD', 
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == 'Rabbit')),
                            'FWD', 
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == 'Lynx')),
                            'FWD', 
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == 'Colt')),
                            'FWD', 
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == 'Jetta')),
                            'FWD', 
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == '240 DL/GL/Turbo')),
                            'RWD', 
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == 'Firebird')),
                            'RWD', 
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == '600')),
                            'FWD', 
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == 'Camaro')),
                            'RWD', 
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == 'Mustang')),
                            'RWD', 
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == 'Conquest')),
                            'RWD', 
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == 'Capri')
                          & (df.year == 1984)),
                            'RWD', 
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == 'Escort Wagon')
                          & (df.year == 1984)),
                            'FWD', 
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == '505 Sedan')
                          & (df.year == 1984)),
                            'RWD', 
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == 'LeBaron')
                          & (df.year == 1984)),
                            'FWD', 
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == 'Colt Vista')),
                            'FWD', 
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == 'Cutlass Supreme')),
                            'FWD', 
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == 'Celebrity Wagon')),
                            'FWD', 
                             df.drive) 

    df['drive'] = np.where(((df.drive.isna() == True)
                          & (df.model == '2000 Sunbird')),
                            'FWD', 
                             df.drive) 

    # Dropping the remaining null values
    df = df.dropna()
    return df.head()

def initial_model_features(df):
    """
    Drops features not used in modeling.
    """
    df = df[['y',
             'year', 
             'make', # drop when modeling
             'model', # drop when modeling'barrels08', 
             'cylinders', 
             'displ', 
             'drive', 
             'trany', 
             'fuelType1',
             'ICE',
             'BEV',
             'PHEV', 
             'VClass', 
             'phevBlended', 
             'guzzler', 
             'tCharger', 
             'sCharger', 
             'startStop' # , 
    #          'co2', # Temp drop for initial modeling
    #          'co2A', # """"""""""""""""""""""""""""""""""""""
    #          'co2TailpipeAGpm', """"""""""""""""""""""""""
    #          'co2TailpipeGpm', """"""""""""""""""""""""""
    #          'pv2', # """"""""""""""""""""""""""""""""""
    #          'lv2', # """"""""""""""""""""""""""""""""""""""""""""""""""
    #          'pv4', # possibly combine these passenger volume features ...
    #          'lv4', # ... with the corresponding luggage volume capacity.
    #          'hpv', # """"""""""""""""""""""""""""""""""""""""""""""""""
    #          'hlv', # Temp drop for modeling to meet deadline

    #          'eng_dscr', # Temp drop for initial modeling
    #          'evMotor', # """"""""""""""""""""""""""""""""""""""""
    #          'trans_dscr', # Temp drop for initial modeling
    #          'phevComb', # PHEV TARGET
            ]]
    return df.head()

